# ENSF 592 - FINAL PROJECT
## Calgary Accident Analysis
### By: Mike Lasby

# README
This notebook is the entry point of our project and functions as the View. The notebook depends on the Controller.py class to retrieve views of our data from the Model. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from geojson import Point, MultiLineString
import geopandas as gpdnum_points
import re
import math
import folium
import matplotlib.pyplot as plt
from controller import Controller

In [15]:
ctrl = Controller()
ctrl.load_data()
ctrl.add_geo_cols()
ctrl.add_cell_col()

Loading Data...
Getting weather at yyc for month 1 in 2018
Getting weather at yyc for month 2 in 2018
Getting weather at yyc for month 3 in 2018
Getting weather at yyc for month 4 in 2018
Getting weather at yyc for month 5 in 2018
Getting weather at yyc for month 6 in 2018
Getting weather at yyc for month 7 in 2018
Getting weather at yyc for month 8 in 2018
Getting weather at yyc for month 9 in 2018
Getting weather at yyc for month 10 in 2018
Getting weather at yyc for month 11 in 2018
Getting weather at yyc for month 12 in 2018
...Data Loaded.
Adding geometry column to speeds from multiline. Flip coords? True
Adding geometry column to volumes from multilinestring. Flip coords? True
Adding geometry column to incidents from location. Flip coords? False
Adding geometry column to cameras from None. Flip coords? True
Adding geometry column to signals from Point. Flip coords? True
Adding geometry column to signs from POINT. Flip coords? True
Adding geometry column to cells from cell_bounds.

In [17]:
ctrl.generate_map()

%%html
<iframe src="index.html" frameborder = "0" width = "960" height = "600" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

SyntaxError: invalid syntax (<ipython-input-17-96598967bc7c>, line 4)

In [18]:
# #HEATMAP
# ctrl.gen_heatmap()

from folium.plugins import HeatMap

def gen_heatmap():
    mapa = ctrl.get_map()
    df = ctrl.get_frame('volumes')
    # df = self.get_frame('volumes')
    data = []  # lat, lng, weight
    # TODO: https://github.com/python-visualization/folium/issues/1271

    for _, row in df.iloc[:10].iterrows():
        volume = row['VOLUME']
        geometry = row['geometry']['coordinates']
        for points in geometry:
            lat = float(points[0])
            lon = float(points[1])
            data.append([lat, lon])
            heat_map = HeatMap(data, name="Volume")
            heat_map.add_to(ctrl.mapa)
 
gen_heatmap()




AttributeError: 'Controller' object has no attribute 'mapa'

In [3]:
speeds = ctrl.get_frame('speeds')
cells = ctrl.get_frame('cells')
# display(speeds)
# display(cells)


def get_avg_speed(cell_idx):
    # print(f'testing cell {cell_idx}')
    speed_sum = 0
    num_points = 0 
    for idx, row in speeds[['cell', 'SPEED']].iterrows():
        cell_dict = row['cell']
        speed = row['SPEED']
        if cell_idx in cell_dict:
            # print(f'{cell_idx} is in {cell_dict}')
            these_points = cell_dict[cell_idx]
            # print(f'these points = {these_points}')
            speed_sum += speed*these_points
            num_points += these_points
    if num_points == 0:
        return np.nan
        # return 0
    return speed_sum/num_points
    
i = 0 
avg_speed = []
while i < 100: 
    avg_speed.append(get_avg_speed(i))
    i+=1
print(avg_speed)
cells['avg_speed'] = avg_speed


[nan, nan, nan, nan, nan, 59.673202614379086, 66.20689655172414, 80.44554455445545, nan, nan, nan, nan, 70.0, 70.45454545454545, 62.04255319148936, 63.6741519350215, 71.49141630901288, 67.21440397350993, 70.76843198338526, 76.34146341463415, nan, nan, nan, 72.1853388658368, 64.78494623655914, 62.06577119036591, 66.55727155727156, 69.90340620233859, 64.29906542056075, 58.07531380753138, nan, nan, nan, 60.0, 62.34993614303959, 62.788844621513945, 68.22074566260613, 70.02633889376646, 72.98299845440495, nan, nan, 73.96825396825396, 65.70743405275779, 73.4090909090909, 59.4337899543379, 58.44332175560467, 70.82397003745318, 62.31227651966627, 72.13842975206612, 80.0, nan, 60.0, 61.02028639618138, 66.2549537648613, 57.83313325330132, 44.25901201602136, 74.37106056453823, 75.69165143205363, 69.5031055900621, 80.0, 110.0, 70.859375, 63.483365949119374, 65.62115621156211, 61.7911227154047, 61.484848484848484, 67.25333333333333, 59.10411622276029, 71.12003780718337, nan, nan, 61.75438596491228,

In [4]:
df=ctrl.get_frame('cells')
df

,cells,cell_bounds,avg_speed
0,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.315796], [50.8797823, -114.2...",NaN
1,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.2702069], [50.8797823, -114....",NaN
2,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.2246178], [50.8797823, -114....",NaN
3,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.1790287], [50.8797823, -114....",NaN
4,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.1334396], [50.8797823, -114....",NaN
...,...,...,...
95,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.175464700000006, -114.0878505], [51.21242...",75.049505
96,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.175464700000006, -114.0422614], [51.21242...",88.552381
97,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.175464700000006, -113.9966723], [51.21242...",69.622980
98,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.175464700000006, -113.9510832], [51.21242...",98.833333


In [5]:
df= ctrl.get_frame('volumes')

cells = ctrl.get_frame('cells')


def get_cell_vol(cell_idx):
    # print(f'testing cell {cell_idx}')
    volume_sum = 0
    num_points = 0 
    for idx, row in df[['cell', 'VOLUME']].iterrows():
        cell_dict = row['cell']
        volume = row['VOLUME']
        if cell_idx in cell_dict:
            # print(f'{cell_idx} is in {cell_dict}')
            these_points = cell_dict[cell_idx]
            # print(f'these points = {these_points}')
            volume_sum += volume
            num_points += these_points
    if num_points == 0:
        return np.nan
        # return 0
    return volume_sum
    
i = 0 
volume_sum = []
while i < 100: 
    volume_sum.append(get_cell_vol(i))
    i+=1
print(volume_sum)
cells['volume_sum'] = volume_sum



[nan, nan, nan, nan, 2000, 98000, 172000, 140000, nan, nan, nan, nan, 14000, 40000, 365000, 1025000, 226000, 930000, 394000, 9000, nan, nan, nan, 75000, 395000, 911000, 848000, 972000, 231000, nan, nan, nan, nan, 6000, 564000, 880000, 1423000, 473000, 202000, 34000, nan, 34000, 165000, 644000, 1143000, 1451000, 1047000, 385000, 196000, nan, nan, 8000, 456000, 824000, 1843000, 3802000, 1452000, 1066000, 437000, 16000, 44000, 249000, 352000, 856000, 1158000, 1267000, 2002000, 1301000, 680000, nan, nan, 427000, 868000, 804000, 316000, 1184000, 368000, 490000, 399000, nan, nan, 180000, 386000, 614000, 887000, 719000, 822000, 313000, 124000, nan, nan, 7000, 14000, 42000, 36000, 23000, 167000, 41000, 40000, nan]


In [6]:
def count_incidents(cell_idx):
    incidents = ctrl.get_frame('incidents')
    # print(f'testing cell {cell_idx}')
    counter = 0
    for _, cell in incidents['cell'].items(): 
        if cell_idx == cell:
            counter +=1
    return counter

i = 0 
inc_count = []

while i<100: 
    inc_count.append(count_incidents(i))
    i+=1
print(inc_count)
cells['incident_count'] = inc_count

display(cells.sort_values(by='incident_count', ascending=False))

[0, 0, 0, 0, 0, 7, 17, 13, 0, 0, 0, 0, 1, 6, 50, 111, 27, 104, 35, 2, 0, 0, 0, 9, 46, 154, 96, 93, 27, 0, 0, 0, 0, 0, 88, 225, 149, 104, 38, 0, 0, 1, 15, 69, 303, 307, 297, 63, 36, 0, 0, 3, 45, 107, 326, 465, 362, 259, 84, 5, 5, 14, 44, 123, 161, 238, 354, 290, 127, 0, 0, 26, 67, 93, 23, 161, 25, 80, 81, 0, 0, 10, 32, 84, 79, 94, 55, 63, 26, 0, 0, 0, 0, 11, 2, 5, 22, 20, 1, 0]


,cells,cell_bounds,avg_speed,volume_sum,incident_count
55,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.0878505], [51.06458...",44.259012,3802000.0,465
56,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.0422614], [51.06458...",74.371061,1452000.0,362
66,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.0645838, -114.0422614], [51.1015441, -113...",67.253333,2002000.0,354
54,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.1334396], [51.06458...",57.833133,1843000.0,326
45,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -114.0878505], [51.0276235000000...",58.443322,1451000.0,307
...,...,...,...,...,...
39,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.953702899999996, -113.9054941], [50.99066...",NaN,34000.0,0
40,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -114.315796], [51.02762350000000...",NaN,NaN,0
49,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -113.9054941], [51.0276235000000...",80.000000,NaN,0
1,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.2702069], [50.8797823, -114....",NaN,NaN,0


In [7]:
df = ctrl.get_frame('temporal')
df
display(df['date'].iloc[0].hour)

df[(df['date'] > '2018-01-01 00:00:00') & (df['date'] < '2018-01-02 00:00:00')]

filt = (df['date'] > '2018-01-01 00:00:00') & (df['date'] < '2018-01-02 00:00:00')
print(filt)

6

0        True
1        True
2        True
3        True
4        True
        ...  
8746    False
8747    False
8748    False
8749    False
8750    False
Name: date, Length: 8751, dtype: bool


In [8]:
df = ctrl.get_frame('temporal')
# display(df)



# display(df['Temp (C)'])

def melt_freeze(temps, freeze_temp, target = 8):
    '''
    return pd.Series bool filter of  hours from 2018 such that Ti is an hour where the temperature fell below freeze_temp (C). Targets Ti are the ith hours after the freeze_temp occured. 

    :Params:    temps: pd.Series of hourly temperatures degrees C
                freeze_temp: temperature of interest 
                target: number of temperatures to capture after the freeze_temp. 
    :Returns:  pd.Series bool filter
    '''
    then = 0
    now = 1
    freeze = freeze_temp

    #list of target hours
    hours=[]

    #pivot + 8 hours
    target = target 

    #  Assume typical dangerous freeze near midnight and we wish to capture the morning     rush hour between 6-8am . 
    
    while then < temps.size: 
        target_hours = []
        try: 
            if temps[now] <= freeze and temps[then] > freeze:
                # print('hit')
                idx = 0
                while idx < target: 
                    target_hours.append(now+idx)
                    idx+=1
                hours.append(target_hours)
            then +=1
            now +=1
        except: 
            then +=1
            now +=1
            continue

    mask_indices =[]
    for targets in hours: 
        for target in targets: 
            if target in mask_indices: 
                continue
            else: 
                mask_indices.append(target)

    mask = []
    idx = 0
    while idx < temps.size:
        if idx in mask_indices:
            mask.append(True)
        else:
            mask.append(False)
        idx += 1
    mask = pd.Series(mask)
    print(f'There were {len(hours)} melt-freeze cycles in 2018!')
    return(mask)

mask = melt_freeze(df['Temp (C)'], 0, 8)


df['date'].loc[mask]







There were 143 melt-freeze cycles in 2018!


35     2018-01-02 17:00:00
36     2018-01-02 18:00:00
37     2018-01-02 19:00:00
38     2018-01-02 20:00:00
39     2018-01-02 21:00:00
               ...        
8712   2018-12-30 06:00:00
8713   2018-12-30 07:00:00
8714   2018-12-30 08:00:00
8715   2018-12-30 09:00:00
8716   2018-12-30 10:00:00
Name: date, Length: 1018, dtype: datetime64[ns]

In [9]:
df =ctrl.get_frame('temporal')

df['date'] = pd.to_datetime(df['Date/Time'])
# display(df)

df2 = ctrl.get_frame('incidents')

# display(df2)
df2['date'] = pd.to_datetime(df2['START_DT'])
# display(df2)

df2['date'] = pd.to_datetime(df2['START_DT'])
mask_2018 = df2['date'].dt.year == 2018
incidents_2018 = df2[mask_2018]
# display(incidents_2018)
# display(df2['date'])


incidents_2018 = incidents_2018.resample('H', on='date')['Count'].count()
display(incidents_2018)
print(incidents_2018.name)
incidents_2018.name = 'incidents'
print(incidents_2018.name)


merged = pd.merge(df, incidents_2018, on = 'date')

# display(merged)

print(merged['Visibility (km)'].unique())

merged = merged.sort_values(by='incidents', ascending = False)
# display(merged)

grouped_weather = merged.groupby(by='Weather')

# display(grouped_weather['Count'].count())

# display(cell_groups['BLADE_TYPE'].agg('count').sort_values(ascending=False).head(10))


# cell_groups = signs_test_df.groupby(by='cell', as_index=True, sort=True)





date
2018-01-01 06:00:00    2
2018-01-01 07:00:00    0
2018-01-01 08:00:00    0
2018-01-01 09:00:00    3
2018-01-01 10:00:00    1
                      ..
2018-12-31 16:00:00    1
2018-12-31 17:00:00    2
2018-12-31 18:00:00    1
2018-12-31 19:00:00    0
2018-12-31 20:00:00    2
Freq: H, Name: Count, Length: 8751, dtype: int64

Count
incidents
[24.1 64.4 32.2 19.3 48.3  9.7  2.4  0.4  0.2  0.8  3.2  6.4  8.1  4.8
  3.6  1.6 16.1 12.9 40.2 56.3 11.3  2.   4.   1.  80.5  nan  2.8  1.2
 14.5  0.6  0.  22.5 20.9 72.4 17.7]


KeyError: 'incidents'

In [10]:
df = ctrl.get_frame('incidents')

filt = df['Count'] != 1

df.loc[filt]

,INCIDENT INFO,DESCRIPTION,START_DT,MODIFIED_DT,QUADRANT,Longitude,Latitude,location,Count,id,date,geometry,cell
